In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [101]:
fam = 'first_active_month'
train_test = utils.read_df_pkl('../input/base_term*0*').set_index(key)[[target, fam]]
sys.exit()
df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:00<00:00, 54.27it/s]


SystemExit: 

In [102]:
# from time_utils import date_add_days

# hist_max_date = df_hist.groupby(key)['purchase_date'].max()
# hist_min_date = df_hist.groupby(key)['purchase_date'].min()
# hist_max_date.name = 'hist_purchase_date_max'
# hist_min_date.name = 'hist_purchase_date_min'
# hist_max_date = hist_max_date.to_frame()
# hist_min_date = hist_min_date.to_frame()

# # month_maxは翌月の数字にする
# hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
# hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_month_max'].map(lambda x: str(x)[:7])

# hist_min_date['hist_purchase_month_min'] = hist_min_date['hist_purchase_date_min'].map(lambda x: str(x)[:7])


# new_max_date = df_new.groupby(key)['purchase_date'].max()
# new_min_date = df_new.groupby(key)['purchase_date'].min()
# new_max_date.name = 'new_purchase_date_max'
# new_min_date.name = 'new_purchase_date_min'
# new_max_date = new_max_date.to_frame()
# new_min_date = new_min_date.to_frame()

# # month_maxは翌月の数字にする
# new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
# new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_month_max'].map(lambda x: str(x)[:7])
# new_min_date['new_purchase_month_min'] = new_min_date['new_purchase_date_min'].map(lambda x: str(x)[:7])

    

train_test = train_test.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)

train_test['hist_purchase_month_max'] = pd.to_datetime(train_test['hist_purchase_month_max'])
train_test['hist_purchase_month_min'] = pd.to_datetime(train_test['hist_purchase_month_min'])
train_test['new_purchase_month_max'] = pd.to_datetime(train_test['new_purchase_month_max'])
train_test['new_purchase_month_min'] = pd.to_datetime(train_test['new_purchase_month_min'])

train_test['first_active_month'].fillna('2017-03', inplace=True)
train_test['first_active_month'] = pd.to_datetime(train_test['first_active_month'])

# personal term
train_test['hist_personal_term']= (train_test['hist_purchase_month_max'].map(lambda x: x.year) - train_test['hist_purchase_month_min'].map(lambda x: x.year)) * 12 + (train_test['hist_purchase_month_max'].map(lambda x: x.month) - train_test['hist_purchase_month_min'].map(lambda x: x.month))
train_test['new_personal_term']= (train_test['new_purchase_month_max'].map(lambda x: x.year) - train_test['new_purchase_month_min'].map(lambda x: x.year)) * 12 + (train_test['new_purchase_month_max'].map(lambda x: x.month) - train_test['new_purchase_month_min'].map(lambda x: x.month))

train_test['hist_regist_term']= (train_test['hist_purchase_month_max'].map(lambda x: x.year) - train_test['first_active_month'].map(lambda x: x.year)) * 12 + (train_test['hist_purchase_month_max'].map(lambda x: x.month) - train_test['first_active_month'].map(lambda x: x.month))
train_test['new_regist_term']= (train_test['new_purchase_month_max'].map(lambda x: x.year) - train_test['first_active_month'].map(lambda x: x.year)) * 12 + (train_test['new_purchase_month_max'].map(lambda x: x.month) - train_test['first_active_month'].map(lambda x: x.month))

col_term = 'hist_regist_term'

train_test['hist_regist_term'] = train_test['hist_regist_term'].map(lambda x:
                                                                    24 if x>=24 else
#                                                                     18 if x>=18 else
#                                                                     15 if x>=15 else
#                                                                     12 if 9<=x and x<=14 else
#                                                                     6 if 6<=x and x<=8  else 
#                                                                     5 if x==5 else
#                                                                     4 if x==4 else
#                                                                     3 if x<=3 
                                                                    x
                                                                   )
train_test['new_regist_term'] = train_test['new_regist_term'].map(lambda x:
                                                                    24 if x>=24 else
                                                                    18 if x>=18 else
#                                                                     15 if x>=15 else
#                                                                     12 if x>=12 else
                                                                    x
                                                                   )


# base = train_test[[target, 'first_active_month', 'hist_personal_term', 'hist_regist_term', 'new_personal_term', 'new_regist_term']].reset_index()
# base.drop('index', inplace=True , axis=1)
base = train_test.reset_index()
utils.to_df_pkl(df=base, path='../input', fname='base_term')

3it [00:00, 23.32it/s]

shape: (325540, 15)


In [147]:
no_out_id_list = []
no_out_id_list += base[base['hist_purchase_month_max']<='2017-05-01'][base['hist_regist_term']<18][key].values.tolist()
no_out_id_list += base[base['hist_regist_term']<=4][key].values.tolist()
list(set(no_out_id_list))
base.loc[base[key].isin(no_out_id_list), 'no_out_flg'] = 1
utils.to_df_pkl(df=base, path='../input', fname='base_term')

5409


SystemExit: 

In [150]:
test = base[base[target].isnull()]
test['no_out_flg'].fillna(0).value_counts()

0.0    101969
1.0     21654
Name: no_out_flg, dtype: int64

In [10]:
df_hist.set_index(key, inplace=True)
df_new.set_index(key, inplace=True)
term_cols = [col for col in train_test.columns if col.count('term')]
df_hist = df_hist.join(train_test[term_cols])
df_new = df_new.join(train_test[term_cols])
df_hist['category_3'] = df_hist['category_3'].map(lambda x: 1 if x=='B' else 0)
df_new['category_3'] = df_new['category_3'].map(lambda x: 1 if x=='B' else 0)
df_new.head()

,authorized_flag,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,first_active_month,all_term,yyyymm,yyyymmdd,weekofyear,yyyy_week,purchase_amount_over,hist_personal_term,hist_regist_term,new_personal_term,new_regist_term,diff_new_hist_regist_term
card_id,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,1,76,0,1.0,B,222,M_ID_08f01305af,2,0.075195,2018-04-03 11:13:35,3,2,21,2017-02-01,454,2018-04,2018-04-03,14,2018-14,0.0,13,12.0,1.0,15.0,3.0
C_ID_00007093c1,1,69,0,1.0,B,879,M_ID_00a6ca8a8a,2,0.090332,2018-04-09 16:23:59,1,9,29,2017-02-01,454,2018-04,2018-04-09,15,2018-15,0.0,13,12.0,1.0,15.0,3.0
C_ID_0001238066,1,314,0,1.0,B,307,M_ID_7d8102bb34,1,0.014168,2018-03-01 16:48:27,1,9,19,2017-09-01,242,2018-03,2018-03-01,9,2018-09,0.0,6,6.0,2.0,8.0,2.0
C_ID_0001238066,1,314,0,1.0,B,367,M_ID_235e546dcc,1,0.074707,2018-03-03 22:44:57,1,9,16,2017-09-01,242,2018-03,2018-03-03,9,2018-09,0.0,6,6.0,2.0,8.0,2.0
C_ID_0001238066,1,333,0,1.0,B,783,M_ID_a88790a464,1,0.105469,2018-03-04 13:05:16,1,9,19,2017-09-01,242,2018-03,2018-03-04,9,2018-09,0.0,6,6.0,2.0,8.0,2.0


In [76]:
# hist , newからauth, category_1, category_3_Bの情報をとってくる

train_test = train_test[[target, 'hist_purchase_date_min', 'first_active_month', 'hist_personal_term', 'hist_regist_term', 'new_personal_term', 'new_regist_term']]
df_list = [df_hist, df_new]
fname_list = ['new', 'hist']

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df, fname in zip(df_list, fname_list):

    aggs = {}
    agg_cols = [key, 'authorized_flag', 'category_1', 'category_3', 'purchase_amount', 'purchase_date']
    if fname.count('hist'):
        aggs['authorized_flag'] = ['mean']
    aggs['category_1'] = ['mean']
    aggs['category_3'] = ['mean']
    aggs['purchase_amount'] = ['sum']
    
    df_agg = df.reset_index()[agg_cols].groupby(key).agg(aggs)
    new_cols = get_new_columns(fname, aggs)
    df_agg.columns = new_cols
    
    train_test = train_test.join(df_agg)

In [87]:
# first_buy
train_test['first_buy'] = (train_test['hist_purchase_date_min'] - train_test['first_active_month']).dt.days//30
train_test['first_buy'] = train_test['first_buy'].map(lambda x:
                                                      24 if x>=24 else 
                                                      2 if x<=2 else 
                                                      x )
print(train_test['first_buy'].dropna().shape)

(325540,)


In [78]:
# Outlier Prob
clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean', 'pred_std']].set_index(key)
train_test = train_test.join(clf)
train_test.head()

,target,hist_purchase_date_min,first_active_month,hist_personal_term,hist_regist_term,new_personal_term,new_regist_term,new_category_1_mean,new_category_3_mean,new_purchase_amount_sum,hist_authorized_flag_mean,hist_category_1_mean,hist_category_3_mean,hist_purchase_amount_sum,first_buy,pred_mean,pred_std
card_id,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-0.820283,2017-06-27 14:18:08,2017-06-01,9,9,2.0,11.0,0.000000,0.015385,27.218750,1.0,0.000000,0.000000,3.939453,2,0.000426,0.000176
C_ID_3d0044924f,0.392913,2017-01-06 16:29:42,2017-01-01,13,13,2.0,15.0,0.088571,0.788571,47.343750,1.0,0.000000,1.000000,0.126953,2,0.008079,0.001434
C_ID_d639edf6cd,0.688056,2017-01-11 08:21:22,2016-08-01,14,18,1.0,18.0,0.000000,0.000000,2.949219,1.0,0.000000,0.000000,0.046875,5,0.004211,0.001213
C_ID_186d6a6901,0.142495,2017-09-26 16:22:21,2017-09-01,6,6,2.0,8.0,0.155844,0.883117,7.828125,1.0,0.142857,0.857143,0.574707,2,0.000789,0.000206
C_ID_cdbd2c0db2,-0.159749,2017-11-12 00:00:00,2017-11-01,4,4,2.0,6.0,0.112782,0.947368,26.359375,1.0,0.055556,0.944444,6.968750,2,0.000259,0.000110
